In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("patstat_pat_no.csv")

In [3]:
data.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted
0,574528.0,307102421.0,1985-04-10,EP,Y
1,574586.0,387396319.0,1987-06-08,EP,Y
2,574660.0,387398325.0,1986-08-04,EP,Y
3,1226456.0,303518198.0,1991-08-29,EP,Y
4,1226485.0,302640285.0,1992-08-12,EP,Y


In [4]:
import mysql.connector

In [5]:
import os
from dotenv import load_dotenv

In [6]:
load_dotenv()
db_config = {
    'host': os.getenv('DB_HOST'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'database': os.getenv('DB_DATABASE')
}

In [ ]:
#for each of the representative patent fetch publn_auth, publn_nr and publn_kind to form key that can be passed into logic mill.
conn = mysql.connector.connect(**db_config)
pat_publn_ids=tuple(data['earliest_pat_publn_id'])
cursor = conn.cursor(dictionary=True)
query = f"""
SELECT    
	t1.pat_publn_id,
    t1.publn_auth,
    t1.publn_nr,
    t1.publn_kind
FROM
    patstat_s24.tls211_pat_publn t1
WHERE
    t1.pat_publn_id IN {pat_publn_ids};
"""
cursor.execute(query)
results = cursor.fetchall()
cursor.close()

True

In [8]:
data2=pd.DataFrame(results)

In [9]:
data2.head()

,pat_publn_id,publn_auth,publn_nr,publn_kind
0,312062057,EP,0000003,A1
1,311747672,EP,0000004,A1
2,311822150,EP,0000031,A1
3,311825145,EP,0000040,A1
4,312062132,EP,0000067,A1


In [10]:
data2[data2["pat_publn_id"]==307102421]

,pat_publn_id,publn_auth,publn_nr,publn_kind
24584,307102421,EP,0286634,A1


In [11]:
data.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted
0,574528.0,307102421.0,1985-04-10,EP,Y
1,574586.0,387396319.0,1987-06-08,EP,Y
2,574660.0,387398325.0,1986-08-04,EP,Y
3,1226456.0,303518198.0,1991-08-29,EP,Y
4,1226485.0,302640285.0,1992-08-12,EP,Y


In [12]:
print(len(data))
print(len(data2))

229282
229282


In [13]:
data.rename(columns={'earliest_pat_publn_id': 'pat_publn_id'}, inplace=True)

In [14]:
data.head()

,docdb_family_id,pat_publn_id,earliest_filing_date,appln_auth,granted
0,574528.0,307102421.0,1985-04-10,EP,Y
1,574586.0,387396319.0,1987-06-08,EP,Y
2,574660.0,387398325.0,1986-08-04,EP,Y
3,1226456.0,303518198.0,1991-08-29,EP,Y
4,1226485.0,302640285.0,1992-08-12,EP,Y


In [15]:
#performing inner join between two datasets
merged_df = pd.merge(data,data2, on=['pat_publn_id'], how='inner')

In [16]:
len(merged_df)

229282

In [17]:
merged_df.head()

,docdb_family_id,pat_publn_id,earliest_filing_date,appln_auth,granted,publn_auth,publn_nr,publn_kind
0,574528.0,307102421.0,1985-04-10,EP,Y,EP,0286634,A1
1,574586.0,387396319.0,1987-06-08,EP,Y,EP,0320505,A1
2,574660.0,387398325.0,1986-08-04,EP,Y,EP,0401206,A1
3,1226456.0,303518198.0,1991-08-29,EP,Y,EP,0670744,A1
4,1226485.0,302640285.0,1992-08-12,EP,Y,EP,0607158,A1


In [19]:
merged_df['key']=merged_df['publn_auth']+merged_df['publn_nr']+merged_df['publn_kind']

In [20]:
merged_df.head()

,docdb_family_id,pat_publn_id,earliest_filing_date,appln_auth,granted,publn_auth,publn_nr,publn_kind,key
0,574528.0,307102421.0,1985-04-10,EP,Y,EP,0286634,A1,EP0286634A1
1,574586.0,387396319.0,1987-06-08,EP,Y,EP,0320505,A1,EP0320505A1
2,574660.0,387398325.0,1986-08-04,EP,Y,EP,0401206,A1,EP0401206A1
3,1226456.0,303518198.0,1991-08-29,EP,Y,EP,0670744,A1,EP0670744A1
4,1226485.0,302640285.0,1992-08-12,EP,Y,EP,0607158,A1,EP0607158A1


In [ ]:
merged_df.to_csv("data_final.csv")

In [74]:
#reading the data which contains the keys that need to be passed to logicmill server
data_final=pd.read_csv("data_final.csv")

In [75]:
data_final.head()

,Unnamed: 0,docdb_family_id,pat_publn_id,earliest_filing_date,appln_auth,granted,publn_auth,publn_nr,publn_kind,key
0,0,574528.0,307102421.0,1985-04-10,EP,Y,EP,286634,A1,EP0286634A1
1,1,574586.0,387396319.0,1987-06-08,EP,Y,EP,320505,A1,EP0320505A1
2,2,574660.0,387398325.0,1986-08-04,EP,Y,EP,401206,A1,EP0401206A1
3,3,1226456.0,303518198.0,1991-08-29,EP,Y,EP,670744,A1,EP0670744A1
4,4,1226485.0,302640285.0,1992-08-12,EP,Y,EP,607158,A1,EP0607158A1


In [76]:
#storing keys in the form of list
keys=list(data_final['key'][0:20])

In [77]:
keys

['EP0286634A1',
 'EP0320505A1',
 'EP0401206A1',
 'EP0670744A1',
 'EP0607158A1',
 'EP0537145A1',
 'EP470080A1',
 'EP0541533A1',
 'EP0580570A1',
 'EP0597835A1',
 'EP0599834A1',
 'EP0555233A1',
 'EP0574583A1',
 'EP0607134A1',
 'EP0602033A1',
 'EP0683881A1',
 'EP0684902A1',
 'EP0688172A1',
 'EP0675992A1',
 'EP0823839A1']

In [78]:
from elasticsearch import Elasticsearch, NotFoundError
d={} #to store the patent and its corresponding k similar patents as key value pairs
patents_not_found=[] #to store patents which were not found when searching in logicmill
embeddings_not_found=[] #to store the patents which were found but for which embeddings were not found

In [79]:
for i in keys:
    client = Elasticsearch(
        hosts=f"https://es2.logic-mill.net:443",
        basic_auth=("elastic", "MTqCwEli2DWAJTFGKwIa"),
        retry_on_status=[500, 501, 502, 503, 504, 524],
        retry_on_timeout=True,
        request_timeout=60,
    )
    try:
        res = client.get(
            index='docdb_cos',
            id=i,
            _source=["embedding"],
        )
        #print('embedding'in res['_source'].keys())
        #print(res)
        if 'embedding'in res['_source'].keys():

            #print(res["_source"]["embedding"])
            embedding = res["_source"]["embedding"]

            k = 10
            query = {
                    "field": "embedding",
                    "num_candidates": min(k * 2, 10_000),
                    "query_vector": embedding
                }
            search = client.search(
                index="docdb_cos",
                knn=query,
                source=False,
                size=k
            )

            print(search["hits"]["hits"])
            l=[] # to store all the similar patents corresponding to the patent with key=i
            for j in search["hits"]["hits"]:
                l.append(j['_id'])
            d[i]=l
                
        else:
            embeddings_not_found.append(i)
    except NotFoundError:
        patents_not_found.append(i)


[{'_index': 'docdb_cos', '_id': 'EP0574583A1', '_score': 0.9991764, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'WO9312941A1', '_score': 0.9973502, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'EP0439049A1', '_score': 0.98186874, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'EP2465692A1', '_score': 0.9802433, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'US4427754A', '_score': 0.97942746, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'EP2409851A2', '_score': 0.9791851, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'EP1813434A1', '_score': 0.97904027, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'US2008152847A1', '_score': 0.9786043, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'US7517833B2', '_score': 0.97850096, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'US2010291326A1', '_score': 0.97843826, '_ig

In [80]:
d

{'EP0574583A1': ['EP0574583A1',
  'WO9312941A1',
  'EP0439049A1',
  'EP2465692A1',
  'US4427754A',
  'EP2409851A2',
  'EP1813434A1',
  'US2008152847A1',
  'US7517833B2',
  'US2010291326A1']}

In [81]:
print(len(d))
print(len(patents_not_found))
print(len(embeddings_not_found))

1
2
17


In [72]:
patents_not_found

['EP470080A1', 'EP0823839A1']

In [73]:
embeddings_not_found

['EP0286634A1',
 'EP0320505A1',
 'EP0401206A1',
 'EP0670744A1',
 'EP0607158A1',
 'EP0537145A1',
 'EP0541533A1',
 'EP0580570A1',
 'EP0597835A1',
 'EP0599834A1',
 'EP0555233A1',
 'EP0607134A1',
 'EP0602033A1',
 'EP0683881A1',
 'EP0684902A1',
 'EP0688172A1',
 'EP0675992A1']

In [44]:
from elasticsearch import Elasticsearch

client = Elasticsearch(
    hosts=f"https://es2.logic-mill.net:443",
    basic_auth=("elastic", "MTqCwEli2DWAJTFGKwIa"),
    retry_on_status=[500, 501, 502, 503, 504, 524],
    retry_on_timeout=True,
    request_timeout=60,
)
res = client.get(
    index='docdb_cos',
    id="EP0574583A1",
    _source=["embedding"],
)
#print('embedding'in res['_source'].keys())
#print(res)
#print(res["_source"]["embedding"])
embedding = res["_source"]["embedding"]

k = 10
query = {
        "field": "embedding",
        "num_candidates": min(k * 2, 10_000),
        "query_vector": embedding
    }
search = client.search(
    index="docdb_cos",
    knn=query,
    source=False,
    size=k
)
a=search["hits"]["hits"]
print(search["hits"]["hits"])

[{'_index': 'docdb_cos', '_id': 'EP0574583A1', '_score': 0.9991764, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'WO9312941A1', '_score': 0.9973502, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'EP0439049A1', '_score': 0.98186874, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'EP2465692A1', '_score': 0.9802433, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'US4427754A', '_score': 0.97942746, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'EP2409851A2', '_score': 0.9791851, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'EP1813434A1', '_score': 0.97904027, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'US2008152847A1', '_score': 0.9786043, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'US7517833B2', '_score': 0.97850096, '_ignored': ['abstract.keyword']}, {'_index': 'docdb_cos', '_id': 'US2010291326A1', '_score': 0.97843826, '_ig